In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import *

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.backend import clear_session
from keras.optimizers import Adam

In [465]:
train = pd.read_csv('/content/drive/MyDrive/kaggle/train.csv')
test = pd.read_csv('/content/drive/MyDrive/kaggle/test.csv')

In [466]:
train['Date']=pd.to_datetime(train['timestamp'])
train['Week'] = train['Date'].dt.week
train['Year'] = train['Date'].dt.year
train['Day'] = train['Date'].dt.day

test['Date']=pd.to_datetime(test['timestamp'])
test['Week'] = test['Date'].dt.week
test['Year'] = test['Date'].dt.year
test['Day'] = test['Date'].dt.day

<ipython-input-466-ecf5abb077ed>:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train['Week'] = train['Date'].dt.week
<ipython-input-466-ecf5abb077ed>:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test['Week'] = test['Date'].dt.week


In [467]:
train['timestamp']=train['timestamp'].str.slice(10, 13)
test['timestamp']=test['timestamp'].str.slice(10, 13)

In [468]:
train['timestamp']=train['timestamp'].astype('int64')
test['timestamp']=test['timestamp'].astype('int64')

In [469]:
train['timestamp'].describe()

count    100000.000000
mean          0.415730
std           0.609658
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           2.000000
Name: timestamp, dtype: float64

In [470]:
test.head(10)

,Unnamed: 0,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,Date,Week,Year,Day
0,0,0,-1.000957,-0.170691,0.124889,-0.979561,0.003150,-0.264673,2000-01-01 00:00:42.700,52,2000,1
1,1,0,-0.874830,0.132696,-0.501727,-1.274911,0.045122,0.121270,2000-01-01 00:28:38.540,52,2000,1
2,2,0,-1.219112,0.074678,0.435331,-0.860820,0.222740,0.008689,2000-01-01 00:07:23.900,52,2000,1
3,3,1,-0.907752,-0.171816,0.211507,-0.972017,0.337799,1.013534,2019-01-12 01:58:44.580,2,2019,12
4,4,0,-1.031261,0.000340,-0.091693,-0.217434,-0.323466,0.931614,2019-01-12 00:59:32.380,2,2019,12
5,5,1,-1.361082,0.246863,0.027296,-0.792353,-0.307068,-1.269952,2000-01-01 01:33:24.080,52,2000,1
6,6,0,-1.002187,-0.000332,-0.212422,-0.087998,-0.317209,0.939888,2019-01-12 00:59:46.880,2,2019,12
7,7,0,-0.358050,-0.166092,0.064071,-1.590621,1.011644,0.834162,2000-01-01 00:25:53.200,52,2000,1
8,8,1,-1.361892,0.028899,0.045418,-0.959959,-1.053144,1.029606,2000-01-01 01:59:11.520,52,2000,1
9,9,2,-0.939794,-0.284829,-0.004960,-0.904230,0.044304,1.187507,2019-01-12 02:00:21.920,2,2019,12


In [407]:
train.shape

(100000, 10)

In [471]:
train.describe()

,Unnamed: 0,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label,Week,Year,Day
count,100000.000000,100000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,90000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,49999.500000,0.415730,-0.876639,-0.110566,-0.013331,-0.780543,0.116559,0.167377,5.945630,36.313500,2005.960870,4.451030
std,28867.657797,0.609658,0.464709,0.348463,0.361989,0.817925,0.593224,0.785482,3.333515,23.200519,8.816197,5.104114
min,0.000000,0.000000,-5.132823,-1.949310,-1.284750,-7.483251,-5.466767,-7.208220,1.000000,2.000000,2000.000000,1.000000
25%,24999.750000,0.000000,-1.016718,-0.181035,-0.281950,-1.050645,-0.190546,-0.243964,3.000000,2.000000,2000.000000,1.000000
50%,49999.500000,0.000000,-0.958268,-0.047425,-0.063756,-0.918144,0.023954,0.113276,7.000000,52.000000,2000.000000,1.000000
75%,74999.250000,1.000000,-0.764307,0.060235,0.222074,-0.157112,0.319651,0.525970,9.000000,52.000000,2019.000000,12.000000
max,99999.000000,2.000000,1.849398,2.413866,2.584467,5.372528,7.182237,6.766558,11.000000,52.000000,2019.000000,12.000000


In [472]:
train.drop('Unnamed: 0', axis=1, inplace = True)
test.drop('Unnamed: 0', axis=1, inplace = True)
train.drop('Date', axis=1, inplace = True)
test.drop('Date', axis=1, inplace = True)

위치에 따라 행동이 비슷하지않을까해서 라벨에 따른 위치값의 평균으로 결측치 채움

In [473]:
def nan(n):
  mean1 = train['A_x'].loc[train['label']==n].mean()
  mean2 = train['A_y'].loc[train['label']==n].mean()
  mean3 = train['A_z'].loc[train['label']==n].mean()
  mean4 = train['B_x'].loc[train['label']==n].mean()
  mean5 = train['B_y'].loc[train['label']==n].mean()
  mean6 = train['B_z'].loc[train['label']==n].mean()
  train['A_x'].loc[train['label']==n] = train['A_x'].loc[train['label']==n].fillna(mean1)
  train['A_y'].loc[train['label']==n] = train['A_y'].loc[train['label']==n].fillna(mean2)
  train['A_z'].loc[train['label']==n] = train['A_z'].loc[train['label']==n].fillna(mean3)
  train['B_x'].loc[train['label']==n] = train['B_x'].loc[train['label']==n].fillna(mean4)
  train['B_y'].loc[train['label']==n] = train['B_y'].loc[train['label']==n].fillna(mean5)
  train['B_z'].loc[train['label']==n] = train['B_z'].loc[train['label']==n].fillna(mean6)

In [474]:
for i in range(1,12,1):
  nan(i)

<ipython-input-473-2e6fc4615032>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['A_x'].loc[train['label']==n] = train['A_x'].loc[train['label']==n].fillna(mean1)
<ipython-input-473-2e6fc4615032>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['A_y'].loc[train['label']==n] = train['A_y'].loc[train['label']==n].fillna(mean2)
<ipython-input-473-2e6fc4615032>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['A_z'].loc

In [475]:
test.head(10)

,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,Week,Year,Day
0,0,-1.000957,-0.170691,0.124889,-0.979561,0.003150,-0.264673,52,2000,1
1,0,-0.874830,0.132696,-0.501727,-1.274911,0.045122,0.121270,52,2000,1
2,0,-1.219112,0.074678,0.435331,-0.860820,0.222740,0.008689,52,2000,1
3,1,-0.907752,-0.171816,0.211507,-0.972017,0.337799,1.013534,2,2019,12
4,0,-1.031261,0.000340,-0.091693,-0.217434,-0.323466,0.931614,2,2019,12
5,1,-1.361082,0.246863,0.027296,-0.792353,-0.307068,-1.269952,52,2000,1
6,0,-1.002187,-0.000332,-0.212422,-0.087998,-0.317209,0.939888,2,2019,12
7,0,-0.358050,-0.166092,0.064071,-1.590621,1.011644,0.834162,52,2000,1
8,1,-1.361892,0.028899,0.045418,-0.959959,-1.053144,1.029606,52,2000,1
9,2,-0.939794,-0.284829,-0.004960,-0.904230,0.044304,1.187507,2,2019,12


In [476]:
train.isna().sum()

timestamp    0
A_x          0
A_y          0
A_z          0
B_x          0
B_y          0
B_z          0
label        0
Week         0
Year         0
Day          0
dtype: int64

In [477]:
train.shape

(100000, 11)

In [490]:
train.head()

,timestamp,A_x,A_y,A_z,B_x,B_y,B_z,label,Week,Year,Day
0,0,-0.259130,-0.834869,-0.485499,0.196409,0.819567,0.384934,8,2,2019,12
1,1,0.370490,0.175042,0.122625,-0.338242,0.358245,0.126491,2,52,2000,1
2,0,-0.257837,-0.881947,-0.391895,0.196027,0.894537,0.411221,8,2,2019,12
3,0,-0.937753,-0.055961,0.362041,-0.929881,0.087673,0.134609,11,52,2000,1
4,0,-0.988320,-0.190390,0.157909,-0.954669,-0.024810,-0.388420,6,52,2000,1


**x y 분리**

In [478]:
target= 'label'

x = train.drop(target, axis = 1)
y = train[target]

In [479]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state=1)

**모델링 :  랜덤포레스트**

In [480]:
model1 = RandomForestClassifier(max_depth=16)
model1.fit(x_train, y_train)

RandomForestClassifier(max_depth=16)

In [483]:
pred1 = model1.predict(x_val)

In [484]:
print('accuracy :',accuracy_score(y_val, pred1))
print('='*60)
print(confusion_matrix(y_val, pred1))
print('='*60)
print(classification_report(y_val, pred1))

accuracy : 0.9117
[[2015   66   20   44   49   14    0    0    0   31   13]
 [ 122 2227    5    0   15    2    0    0    0   53    9]
 [  85    7 1268  144  123    0    0    0    0    1    0]
 [  46    0  125 1291   38    0    0    0    0    0    0]
 [ 158   26  106   88  682    0    0    0    0    1    0]
 [  32    6    0    0    0  970    0    0    0   24   35]
 [   0    0    0    0    0    0 2145    0    0    0    0]
 [   0    0    0    0    0    0    0 2295    0    0    0]
 [   0    0    0    0    0    0    0    0 1877    0    0]
 [  19   26    0    0    0    3    0    0    0 1780   97]
 [   3    1    0    0    0    7    0    0    0  122 1684]]
              precision    recall  f1-score   support

           1       0.81      0.89      0.85      2252
           2       0.94      0.92      0.93      2433
           3       0.83      0.78      0.80      1628
           4       0.82      0.86      0.84      1500
           5       0.75      0.64      0.69      1061
           6      

In [304]:
pred1.shape

(20000,)

In [305]:
pred1 = model1.predict(test)

In [306]:
pred1.shape

(13234,)

**모델링: XGB**

In [485]:
y_train

78689    10
76423     8
86945    11
57427     7
34616     7
         ..
50057     2
98047     7
5192      4
77708     4
98539     6
Name: label, Length: 80000, dtype: int64

In [486]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_xgb = le.fit_transform(y_train)

In [487]:
model2 = XGBClassifier()
model2.fit(x_train, y_train_xgb)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [488]:
pred2 = model2.predict(x_val.values)
pred2 = le.inverse_transform(pred2)

In [489]:
print('accuracy :',accuracy_score(y_val, pred2))
print('='*60)
print(confusion_matrix(y_val, pred2))
print('='*60)
print(classification_report(y_val, pred2))

accuracy : 0.9387
[[2071   50   19   22   43   16    0    0    0   19   12]
 [  73 2303    4    0   12    5    0    0    0   28    8]
 [  48    4 1384  102   90    0    0    0    0    0    0]
 [  25    1   90 1348   36    0    0    0    0    0    0]
 [  85   29   88   49  809    0    0    0    0    1    0]
 [  23    7    0    0    0 1000    0    0    0   13   24]
 [   0    0    0    0    0    0 2144    0    1    0    0]
 [   0    0    0    0    0    0    0 2295    0    0    0]
 [   0    0    0    0    0    0    0    0 1877    0    0]
 [  17   16    0    0    0    6    0    0    0 1822   64]
 [   2    3    0    0    0    8    0    0    0   83 1721]]
              precision    recall  f1-score   support

           1       0.88      0.92      0.90      2252
           2       0.95      0.95      0.95      2433
           3       0.87      0.85      0.86      1628
           4       0.89      0.90      0.89      1500
           5       0.82      0.76      0.79      1061
           6      

**모델링: 디시전트리**

In [451]:
model3 = DecisionTreeClassifier(max_depth=16)
model3.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=16)

In [452]:
pred3 = model3.predict(x_val)

In [453]:
print('accuracy :',accuracy_score(y_val, pred3))
print('='*60)
print(confusion_matrix(y_val, pred3))
print('='*60)
print(classification_report(y_val, pred3))

accuracy : 0.8721
[[1812  120   61   59  127   22    0    0    0   33   18]
 [ 188 2066   19    3   43   12    0    0    0   79   23]
 [ 106    7 1213  157  145    0    0    0    0    0    0]
 [  53    0  170 1218   59    0    0    0    0    0    0]
 [ 158   41  216  110  535    0    0    0    0    1    0]
 [  34    8    0    1    1  960    0    0    0   17   46]
 [   0    0    0    0    0    0 2143    2    0    0    0]
 [   0    0    0    0    0    0    0 2295    0    0    0]
 [   0    0    0    0    0    0    0    0 1877    0    0]
 [  32   50    1    0    4   16    0    0    0 1656  166]
 [   7    5    0    0    0   25    0    0    0  113 1667]]
              precision    recall  f1-score   support

           1       0.76      0.80      0.78      2252
           2       0.90      0.85      0.87      2433
           3       0.72      0.75      0.73      1628
           4       0.79      0.81      0.80      1500
           5       0.59      0.50      0.54      1061
           6      

In [450]:
pred3.shape

(20000,)

In [232]:
param = {'max_depth' : range(1, 51)}

# Grid Search 선언
  # cv=5
  # scoring='accuracy'
model3_1 = GridSearchCV(model3,
                    param,
                    cv=5,
                    scoring='accuracy')

In [237]:
print(model3_1.best_params_)

{'max_depth': 16}


In [233]:
model3_1.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 51)}, scoring='accuracy')

In [234]:
pred3_1 = model3_1.predict(x_val)

In [235]:
print('accuracy :',accuracy_score(y_val, pred3_1))
print('='*60)
print(confusion_matrix(y_val, pred3_1))
print('='*60)
print(classification_report(y_val, pred3_1))

accuracy : 0.85985
[[1827  102   71   39  115   22    1    1   19   27   28]
 [ 175 2003   16    5   49   12    1    5   22  113   32]
 [ 107    7 1183  175  143    0    1    3    4    4    1]
 [  44    1  161 1225   66    0    1    1    0    1    0]
 [ 190   41  184   87  551    1    1    0    2    2    2]
 [  40   12    0    0    1  952    0    0    7   20   35]
 [   1    2    4    1    2    0 2134    0    1    0    0]
 [   0    6    0    0    1    0    0 2279    2    6    1]
 [  21   15    2    0    0    2    1    0 1808   10   18]
 [  40   99    2    0    2   16    0    3   16 1598  149]
 [  20    8    1    0    0   16    0    2   15  118 1637]]
              precision    recall  f1-score   support

           1       0.74      0.81      0.77      2252
           2       0.87      0.82      0.85      2433
           3       0.73      0.73      0.73      1628
           4       0.80      0.82      0.81      1500
           5       0.59      0.52      0.55      1061
           6     

**XGB 선택**

In [442]:
result = model2.predict(test.values)
result = le.inverse_transform(result)

In [443]:
result

array([ 6,  4, 11, ...,  4,  1,  7])

In [444]:
result.shape

(13234,)

In [445]:
sample = pd.read_csv('/content/drive/MyDrive/kaggle/sample.csv')

In [327]:
sample.shape

(13234, 2)

In [446]:
result_df = pd.DataFrame(data = result , columns = ['label'])

In [447]:
result_df.reset_index(names='ID', inplace=True)

In [448]:
result_df.head()

,ID,label
0,0,6
1,1,4
2,2,11
3,3,9
4,4,7


In [449]:
result_df.to_csv('/content/drive/MyDrive/kaggle/result.csv', index = False)